# Shear Likelihood 1.1

### Preparing necessary libraries

In [1]:
import os
import sys
import gi

gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

os.environ['CLMM_MODELING_BACKEND'] = 'nc'

__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
from numpy import random
plt.rcParams['font.family']=['gothambook','gotham','gotham-book','serif']

import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.theory as theory
from clmm import Cosmology
from clmm.support import mock_data as mock
from clmm.utils import convert_units

### Creating mock data

In [2]:
np.random.seed(11)

cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

cluster_m     = 1.e15
logm = np.log(cluster_m)/np.log(10)
cluster_z     = 0.4
concentration = 4
ngals         = 10000
Delta         = 200
cluster_ra    = 0.0
cluster_dec   = 90.0

ideal_data   = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 0.8, zsrc_min = cluster_z + 0.1, shapenoise=0.005, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
ideal_data_z = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo,'chang13', zsrc_min = cluster_z + 0.1, shapenoise=0.005, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
noisy_data_z = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 'chang13', zsrc_min = cluster_z + 0.1, shapenoise=0.005, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)

gc_ideal   = clmm.GalaxyCluster("CL_ideal", cluster_ra, cluster_dec, cluster_z, ideal_data)
gc_ideal_z = clmm.GalaxyCluster("CL_ideal_z", cluster_ra, cluster_dec, cluster_z, ideal_data_z)
gc_noisy_z = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, noisy_data_z)


### Calculating observables

In [3]:
theta_ideal, gt_ideal, gx_ideal       = gc_ideal.compute_tangential_and_cross_components(geometry="flat")
theta_ideal_z, gt_ideal_z, gx_ideal_z = gc_ideal_z.compute_tangential_and_cross_components(geometry="flat")
theta_noisy_z, gt_noisy_z, gx_noisy_z = gc_noisy_z.compute_tangential_and_cross_components(geometry="flat")

# Convert from theta to radius
radius_ideal, radius_ideal_z, radius_noisy_z = [convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=cluster_z, cosmo=cosmo) for gc in [gc_ideal, gc_ideal_z, gc_noisy_z]]

In [4]:
gc_ideal.galcat

ra,dec,e1,e2,z,ztrue,id,theta,et,ex
float64,float64,float64,float64,float64,float64,int64,float64,float64,float64
46.29184640208061,89.81856252394499,0.002612093914611752,-0.003269453109598035,0.8,0.8,0,0.0031666813436682876,0.002612093914611855,-0.003269453109597953
51.52862138011059,89.74823160545624,0.001749494204237913,-0.0065094656730693085,0.8,0.8,1,0.004394187437248733,0.0017494942042380768,-0.006509465673069264
13.525811539094493,89.935489123421,0.026024166310292234,-0.014441727978658005,0.8,0.8,2,0.001125927199651368,0.026024166310292605,-0.014441727978657337
280.07334896586775,89.90628582827307,-0.01624742804700041,0.009668504599943152,0.8,0.8,3,0.0016356208524143064,-0.0162474280469994,0.00966850459994485
169.61950641689276,89.81833621996996,0.013009526604655269,-0.001962058910037325,0.8,0.8,4,0.0031706310931428443,0.013009526604655493,-0.001962058910035836
2.7726261532263266,89.87641960764591,0.00993025951018321,-0.008913464017741934,0.8,0.8,5,0.0021568847374852213,0.009930259510183233,-0.008913464017741908
80.09407872148873,89.7971144556942,-0.0037674209151077093,-0.0045697537518534,0.8,0.8,6,0.003541020752837054,-0.0037674209151074885,-0.004569753751853582
150.78293301065207,89.98938731813249,0.03905111642495602,0.07641201874549532,0.8,0.8,7,0.00018522624105481346,0.039051116424823076,0.07641201874556326
273.70123343891686,89.81838889474838,-0.008856990642359801,0.0015130769889717286,0.8,0.8,8,0.0031697117448267194,-0.008856990642359713,0.001513076988972245


### Binning the data

In [5]:
bin_edges = da.make_bins(0.7, 4, 15, method='evenlog10width')

profile_ideal   = gc_ideal.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)
profile_ideal_z = gc_ideal_z.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)
profile_noisy_z = gc_noisy_z.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)

profiles = [profile_ideal, profile_ideal_z, profile_noisy_z]

/home/caiooliveira/.local/lib/python3.10/site-packages/clmm-1.1.10-py3.10.egg/clmm/utils.py:176: RuntimeWarning: invalid value encountered in sqrt


In [6]:
print(sum(r <= 4. and r >= 0.7 for r in radius_ideal))
print(sum([len(profile_ideal[i]['gal_id']) for i in range(0,15)]))

7597
7597


### Calculating Likelihood

The measured shear $g^i_t$ is given by 

$$ g^i_t = g^{I,i}_t + g^{L,i}_t $$

where $g^{I,i}_t$ is the intrisic shear and $g^{L,i}_t$ is the lensing shear. Assuming that the intrinsic shear is drawn from a normal distribution with average 0 and variance $\sigma_{g^I_t}^2$, the probability of measuring $g^i_t$, given a certain $r$, $z$ and other cosmological parameters $\vec{\theta_c}$ is given by

$$ P(g^i_t | r, z, \vec{\theta_c}) = \frac{1}{\sqrt{2\pi}\sigma_{g^I_t}} \exp\left[\frac{-(g^i_t - g^{L,i}_t(r,z,\vec{\theta_c}))^2}{2\sigma_{g^I_t}^2}\right] $$
Then, to find the likelihood of the set of all $\{g^i_t\}_i^{N}$ measured, we perform the following integral for all $g^i_t$:

$$ P(g^i_t | \vec{\theta_c}) = \int_0^{r_{max}} \int_{z_{cluster}}^{\infty} \mathcal{d}r \mathcal{d}z P(g^i_t | r, z, \vec{\theta_c}) P(r) P(z) $$

where $P(r)$ and $P(z)$ are the distributions for $r$ and $z$.

This is a computationally expensive process though. To avoid it, we bin the data. We then have to perform the same integrals, but for each bin (and their average shear $\bar{g}^i_t$). At this moment we'll choose to not calculate them, and instead use an estimator for the bin likelihood

$$ \mathcal{L}_i = \tilde{P}(\bar{g}^i_t | \vec{\theta_c}) = \frac{1}{N_i} \sum_j^{N_i} P(\bar{g}^i_t | r_j, z_j, \vec{\theta_c}) $$

where $N_i$ is the number of galaxies in the bin. The total likelihood is then

$$ \mathcal{L} = \prod_i^M \mathcal{L}_i = \prod_i^M \left[ \frac{1}{N_i} \sum_j^{N_i} P(\bar{g}^i_t | r_j, z_j, \vec{\theta_c}) \right] $$

and $M$ is the total number of bins.

In [7]:
def shear_probability(gt_obs, gt_th, sigma):

    return np.exp(-np.power(np.subtract(gt_obs, gt_th), 2)/2/sigma**2)/np.sqrt(2*np.pi)/sigma

In [8]:
def shear_log_likelihood(gc, shapenoise, cosmo, sigma=1, m=1.e15, concentration=4, delta=200, model='nfw'):

    n_bins = len(gc.profile)
    cluster_z = gc.z
    likelihood = 0

    gal_z = gc.galcat['z']
    gal_r = convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=cluster_z, cosmo=cosmo)
    gt_th = clmm.compute_reduced_tangential_shear(gal_r, m, concentration, cluster_z, gal_z, cosmo, delta_mdef=200, halo_profile_model=model)

    likelihood = 0

    for i in range(n_bins):

        gal_bin = gc.profile['gal_id'][i]
        gt_th_bin = [gt_th[gal_id] for gal_id in gal_bin]
        avg_gt_obs_bin = gc.profile['gt'][i]

        shear_prob_bin = np.mean(shear_probability(avg_gt_obs_bin, gt_th_bin, shapenoise))
        likelihood += np.log(shear_prob_bin)

    return -2*likelihood

In [9]:
shear_log_likelihood(gc_ideal, 0.005, cosmo)

320.1247629706557